In this notebook we simulate rounded pentagons using the [DEM component](https://hoomd-blue.readthedocs.io/en/stable/module-dem-pair.html#hoomd.dem.pair.WCA) of HOOMD-Blue. We visualize the shapes using the matplotlib backend and color them by their orientation.

In [ ]:
import flowws
import gtar
from hoomd_flowws.Init import Init
from hoomd_flowws.DEMInteraction import DEMInteraction
from hoomd_flowws.Run import Run
from hoomd_flowws.ShapeDefinition import ShapeDefinition

import json
import plato, plato.draw.matplotlib as draw
import freud
import numpy as np
import IPython

In [ ]:
shape = dict(type='regular_ngon', n=5, 
             modifications=[dict(type='scale', factor=2), dict(type='round', radius=.5)])

storage = flowws.DirectoryStorage()
stages = [
    ShapeDefinition(shape_arguments=[shape]),
    Init(number=128),
    DEMInteraction(type='wca'),
    Run(steps=1e3, integrator='langevin'),
    Run(steps=1e5, integrator='langevin', compress_to=.9, dump_period=1e4),
]

flowws.Workflow(stages, storage).run()

In [ ]:
with gtar.GTAR('dump.sqlite', 'r') as traj:
    (posRec, quatRec, boxRec), frames = traj.framesWithRecordsNamed(['position', 'orientation', 'box'])
    
    for (_, type_shapes) in traj.recordsNamed('type_shapes.json'):
        type_shapes = json.loads(type_shapes)
    
    positions = traj.getRecord(posRec, frames[-1])
    orientations = traj.getRecord(quatRec, frames[-1])
    box = traj.getRecord(boxRec, frames[-1])   

shapedef = type_shapes[0]

In [ ]:
# orientation angle
theta = 2*np.arccos(orientations[:, 0])
colors = np.ones((len(positions), 4))
colors[:, :3] = plato.cmap.cubeellipse(5*theta)

prim = draw.Spheropolygons(
    positions=positions[:, :2], orientations=orientations, colors=colors, 
    vertices=shapedef['vertices'], radius=shapedef['rounding_radius'], outline=.1)
scene = draw.Scene(prim, zoom=.7)

target = '../../gallery/flowws_pentagons_matplotlib.png'
scene.save(target)